# [CDAF] Atividade 4

## Nome: Thiago Pádua de Carvalho

## Matrícula: 2020007066

## Referências
- [1] https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
- [2] https://socceraction.readthedocs.io/en/latest/api/generated/socceraction.xthreat.ExpectedThreat.html#socceraction.xthreat.ExpectedThreat
- [3] https://socceraction.readthedocs.io/en/latest/api/generated/socceraction.xthreat.get_successful_move_actions.html#socceraction.xthreat.get_successful_move_actions
- [4] https://socceraction.readthedocs.io/en/latest/documentation/valuing_actions/xT.html

In [60]:
# Importando bibliotecas
from tqdm import tqdm
import numpy as np
import pandas as pd
import socceraction.spadl as spd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from socceraction import xthreat as xt

### LaLiga  p/ SPADL com pré-processamentos

In [2]:
# carregando os eventos
path = f'../data/events/events_Spain.json'
events = pd.read_json(path_or_buf=path)
events

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id
0,8,Simple pass,[{'id': 1801}],3542,"[{'y': 61, 'x': 37}, {'y': 50, 'x': 50}]",2565548,Pass,682,1H,2.994582,85,180864419
1,8,Simple pass,[{'id': 1801}],274435,"[{'y': 50, 'x': 50}, {'y': 30, 'x': 45}]",2565548,Pass,682,1H,3.137020,85,180864418
2,8,Simple pass,[{'id': 1801}],364860,"[{'y': 30, 'x': 45}, {'y': 12, 'x': 38}]",2565548,Pass,682,1H,6.709668,85,180864420
3,8,Simple pass,[{'id': 1801}],3534,"[{'y': 12, 'x': 38}, {'y': 69, 'x': 32}]",2565548,Pass,682,1H,8.805497,85,180864421
4,8,Simple pass,[{'id': 1801}],3695,"[{'y': 69, 'x': 32}, {'y': 37, 'x': 31}]",2565548,Pass,682,1H,14.047492,85,180864422
...,...,...,...,...,...,...,...,...,...,...,...,...
628654,8,Simple pass,[{'id': 1801}],20623,"[{'y': 25, 'x': 66}, {'y': 2, 'x': 88}]",2565927,Pass,682,2H,2939.077491,85,253302671
628655,7,Acceleration,[{'id': 1801}],122832,"[{'y': 2, 'x': 88}, {'y': 21, 'x': 97}]",2565927,Others on the ball,682,2H,2940.515560,70,253302673
628656,8,Cross,"[{'id': 401}, {'id': 1802}]",122832,"[{'y': 21, 'x': 97}, {'y': 26, 'x': 92}]",2565927,Pass,682,2H,2942.098761,80,253302674
628657,8,Simple pass,[{'id': 1801}],40756,"[{'y': 74, 'x': 8}, {'y': 56, 'x': 9}]",2565927,Pass,675,2H,2943.089232,85,253302698


In [3]:
# pré processamento em colunas da tabela de eventos para facilitar a conversão p/ SPADL
events = events.rename(columns={'id': 'event_id', 'eventId': 'type_id', 'subEventId': 'subtype_id',
                                'teamId': 'team_id', 'playerId': 'player_id', 'matchId': 'game_id'})
events['milliseconds'] = events['eventSec'] * 1000
events['period_id'] = events['matchPeriod'].replace({'1H': 1, '2H': 2})

In [4]:
# carregando as partidas, pois vamos saber quais times jogam em casa e fora p/ usar como parametro do SPADL
path = f'../data/matches/matches_Spain.json'
matches = pd.read_json(path_or_buf=path)

In [5]:
# as informações dos times de cada partida estão em um dicionário dentro da coluna 'teamsData', então vamos separar essas informações
team_matches = []
for i in tqdm(range(len(matches))):
    match = pd.DataFrame(matches.loc[i, 'teamsData']).T
    match['matchId'] = matches.loc[i, 'wyId']
    team_matches.append(match)
team_matches = pd.concat(team_matches).reset_index(drop=True)

100%|██████████| 380/380 [00:00<00:00, 1366.09it/s]


In [6]:
# fazendo a conversão p/ SPADL, padronizando a direção de jogo da esquerda p/ a direita e adicionando os nomes dos tipos de ações
spadl = []
game_ids = events.game_id.unique().tolist()
for g in tqdm(game_ids):
    match_events = events.loc[events.game_id == g]
    match_home_id = team_matches.loc[(team_matches.matchId == g) & (team_matches.side == 'home'), 'teamId'].values[0]
    match_actions = spd.wyscout.convert_to_actions(events=match_events, home_team_id=match_home_id)
    match_actions = spd.play_left_to_right(actions=match_actions, home_team_id=match_home_id)
    match_actions = spd.add_names(match_actions)
    spadl.append(match_actions)
spadl = pd.concat(spadl).reset_index(drop=True)

100%|██████████| 380/380 [02:30<00:00,  2.52it/s]


In [7]:
# adicionando o nome dos jogadores
path = f'../data/players.json'
players = pd.read_json(path_or_buf=path)
players['player_name'] = players['firstName'] + ' ' + players['lastName']
players = players[['wyId', 'player_name']].rename(columns={'wyId': 'player_id'})
spadl = spadl.merge(players, on='player_id', how='left')

## Questão 1
- Crei um dataframe "shots" à partir do dataframe "spadl", contendo apenas os chutes.
- Crie 4 colunas no dataframe "shots" a serem usadas como features de um modelo de xG.
- Justifique a escolha das features.

In [29]:
def euclidean_distance(point1, point2=[105, 34]):
    point1 = np.array(point1)
    point2 = np.array(point2)
    return np.sqrt(np.sum((point2 - point1) ** 2)) 

In [42]:
def find_angle_to_goal(point):
    point = np.array(point)  # shot coordinates
    post1 = np.array([105, 37.66])
    post2 = np.array([105, 30.34])

    vector1 = post1 - point
    vector2 = post2 - point

    # Calculates the length of the vectors that goes from the player in the moment of the shot to the posts
    vector1_length = euclidean_distance(point, post1)  
    vector2_length = euclidean_distance(point, post2)

    cos_theta = np.dot(vector1, vector2) / (vector1_length * vector2_length)
    
    return np.arccos(cos_theta) * 180 / np.pi  # Returns the angle in degrees

In [50]:
shots = spadl.loc[spadl.type_name == 'shot'].reset_index(drop=True)
shots.drop(columns=['action_id', 'game_id', 'original_event_id', 'bodypart_id', 'type_id'], inplace=True)

In [59]:
shots['distance'] = shots.apply(lambda row: euclidean_distance([row['start_x'], row['start_y']]), axis=1)
shots['angle'] = shots.apply(lambda row: find_angle_to_goal([row['start_x'], row['start_y']]), axis=1)
shots['distXangle'] = shots['distance'] * shots['angle']
shots['squared_distance'] = shots['distance'] ** 2
shots.tail()

,period_id,time_seconds,team_id,player_id,start_x,start_y,end_x,end_y,result_id,type_name,result_name,bodypart_name,player_name,distance,angle,distXangle,squared_distance,goal
7974,2,1776.027412,682,267134,97.65,23.12,97.65,23.12,0,shot,fail,foot,Roger Beyker Mart\u00ednez Tobinson,13.130000,18.694513,245.458956,172.3969,False
7975,2,1944.188119,682,267134,94.50,46.24,94.50,46.24,0,shot,fail,foot,Roger Beyker Mart\u00ednez Tobinson,16.126612,17.306400,279.093590,260.0676,False
7976,2,2385.837008,682,134174,96.60,51.00,105.00,37.40,1,shot,success,foot,Samuel Castillejo Azuaga,18.962067,10.072200,190.989743,359.5600,True
7977,2,2722.835144,675,3321,86.10,47.60,105.00,30.60,0,shot,fail,foot,Karim Benzema,23.284544,14.662142,341.401296,542.1700,False
7978,2,2857.346465,675,4498,92.40,38.76,105.00,40.80,0,shot,fail,foot,Lucas V\u00e1zquez Iglesias,13.469135,28.763621,387.421100,181.4176,False


### Justificando Features
Para a criação do modelo de xG, foram escolhidas as seguintes features:
- **distance**: Distância entre o chute e o gol. \
Esta feature foi escolhida pois - desconsiderando jogadores no caminho - quanto mais próximo do gol, maior a chance de marcar.

- **angle**: Ângulo entre o chute e o gol. \
O ângulo entre as traves, visto pelo jogador que chuta, é aproximadamente proporcional à probabilidade de gol.

- **distXangle**: Multiplicação da distência pelo ângulo.
- **squared_distance**: Quadrado da distência do chute até o gol \
As duas últimas foram escolhidas como modo de evitar que durante a aplicação de modelos preditivos haja overfitting, isto é, quando o modelo não consegue generalizar e se adequa somente aos dados de treinamento.

## Questão 2
- Crie uma coluna numérica binária "goal" no dataframe "shots" indicando se o chute resultou em gol ou não.
- Use regressão logística [1] p/ treinar (.fit(X_train, y_train)) um modelo de xG usando as features criadas na questão 1.
- Use 70% dos dados para treino e 30% para teste.
- Reporte a acurácia do modelo para os conjuntos de treino (.score(X_train, y_train)) e teste (.score(X_test, y_test)).

In [65]:
shots['goal'] = shots['result_id'].isin([1])
train, test = train_test_split(shots, test_size=0.3, random_state=42)

model = LogisticRegression()
model.fit(train[['distance', 'angle', 'distXangle', 'squared_distance']], train['goal']).coef_

array([[-1.88073206e-01,  1.03294615e-02, -9.63109573e-05,
         1.82462434e-03]])

In [64]:
score = model.score(test[['distance', 'angle', 'distXangle', 'squared_distance']], test['goal'])
print(f"A precisão média observada foi de {score:.2f}")

A precisão média observada foi de 0.89


## Questão 3
- Use o modelo treinado na questão 2 p/ prever a probabilidade de gol de todos os chutes do dataframe "shots". Reporte essas probabilidades no dataframe "shots" em uma coluna "xG".
- Agrupe o dataframe "shots" por "player_name" e reporte a soma dos "goal" e "xG".
- Reporte os 10 jogadores com maior xG.
- Reporte os 10 jogadores com maior diferença de Gols e xG.

## Questão 4 [4]
- Instancie um objeto ExpectedThreat [2] com parâmetros l=25 e w=16.
- Faça o fit do modelo ExpectedThreat com o dataframe "spadl".

## Questão 5
- Crie um dataframe "prog_actions" à partir do dataframe "spadl", contendo apenas as ações de progressão e que são bem-sucedidas [3].
- Use o método rate do objeto ExpectedThreat p/ calcular o valor de cada ação de progressão do dataframe "prog_actions", em uma coluna chamada "action_value".
- Agrupe o dataframe "prog_actions" por "player_name" e reporte a soma dos "action_value".
- Reporte os 10 jogadores com maior "action_value".